## Installing the provided library

In [1]:
! cd ../dist/ && pip3 install QPandas-0.20-py3-none-any.whl && cd ../doc/

/bin/bash: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8)
Processing ./QPandas-0.20-py3-none-any.whl
QPandas is already installed with the same version as the provided wheel. Use --force-reinstall to force an installation of the wheel.


In [2]:
from QPandas.qframe import QFrame, QSeries

## Series
We will provide some simple use cases that reflect the required functionality asked in doc. The API is similar to the Pandas pd.Series(see https://pandas.pydata.org/docs/reference/api/pandas.Series.html). In the next cells we will show the following:
- Creating a series
- Accessing attributes, entries and filtering based on conditions
- Combining the series in a frame

In [5]:
#Setup a common idx for the next series
idx = list(range(4))
#Create series using list (comprehension)
t = QSeries(data= [True  if i%2==0 else False for i in range(len(idx))], index=idx,dtype=bool,name="allbool")
price = QSeries([i**2/2 for i in range(4)],idx,float,name="current_price")
#Series representation
print(t)
print(price)
print(price[t])
print(t[t==True])

'alltrue'
__________________
|  0   |  True  |
__________________
|  1   |  False  |
__________________
|  2   |  True  |
__________________
|  3   |  False  |
__________________
dtype: <class 'bool'>
'current_price'
__________________
|  0   |  0.0  |
__________________
|  1   |  0.5  |
__________________
|  2   |  2.0  |
__________________
|  3   |  4.5  |
__________________
dtype: <class 'float'>
'current_price[alltrue]'
__________________
|  0   |  0.0  |
__________________
|  2   |  2.0  |
__________________
dtype: <class 'float'>
"alltrue[alltrue == <class 'bool'>]"
__________________
|  0   |  True  |
__________________
|  2   |  True  |
__________________
dtype: <class 'bool'>


Similar to pandas, we can create queries based on conditions

In [15]:
#Useful Attributes/ Methods
print(len(price))
print(price.index,price.name,price.dtype)
price.set_name("Old price")
print(price)

4
[0, 1, 2, 3] Old price <class 'float'>
'Old price'
__________________
|  0   |  0.0  |
__________________
|  1   |  0.5  |
__________________
|  2   |  2.0  |
__________________
|  3   |  4.5  |
__________________
dtype: <class 'float'>
'alltrue | alltrue'
__________________
|  0   |  None  |
__________________
|  1   |  False  |
__________________
|  2   |  None  |
__________________
|  3   |  False  |
__________________
dtype: <class 'bool'>


In [17]:
# This shows how one can alter the behaviour of None interaction
# For details, it is worth checking the implementation of operations
t1 = QSeries(data= [True  if i%2==0 else False for i in range(len(idx))], index=idx,dtype=bool,name="alltrue")
t2 = QSeries(data= [None  if i%2==0 else False for i in range(len(idx))], index=idx,dtype=bool,name="alltrue")
print(t1|t2)
QSeries.allow_none_equality = True
print(t1|t2)

'alltrue | alltrue'
__________________
|  0   |  None  |
__________________
|  1   |  False  |
__________________
|  2   |  None  |
__________________
|  3   |  False  |
__________________
dtype: <class 'bool'>
'alltrue | alltrue'
__________________
|  0   |  True  |
__________________
|  1   |  False  |
__________________
|  2   |  True  |
__________________
|  3   |  False  |
__________________
dtype: <class 'bool'>


In [18]:
#Dtype. Running any operation between Series of identical dtype is more or less similar to pandas(though it integrates None entries in that dtype). However, any operation with different dtypes will cause errors. Warnings are generally raised when certain arguments are missing and the indexing/column naming was inferred from data

price[price>1.0] # this works fine
# price[price>1] # this will raise an error. Uncomment to check.

"Old price[Old price > <class 'float'>]"
__________________
|  2   |  2.0  |
__________________
|  3   |  4.5  |
__________________
dtype: <class 'float'>

![](table.png "Alt text" width=400 height=300)


In [19]:
#Let's create